<h2>For quick implement code only.</h2>
Implement model loader and apply prediction base on model. <BR>
** This is only for testing, the real use is predict.py which integrate Flask for web browser interation **

In [1]:
import pandas as pd
import numpy as np

In [2]:
#load data
team_info = pd.read_csv('real_data.csv', header=0, index_col = 7)
team_info_verse = pd.read_csv('verse_data.csv', header=0, index_col = 0)
print team_info.columns

Index([u'RD1_WIN', u'RD2_WIN', u'RD3_WIN', u'RD4_WIN', u'RD5_WIN', u'RD6_WIN',
       u'RD7_WIN', u'TEAM_RATING', u'TEAM_SEED', u'RANK', u'PCT', u'RPI',
       u'PCT_NC', u'RPI_NC', u'RANK_NC', u'PCT_C', u'RPI_C', u'RANK_OLD',
       u'ROAD', u'NC', u'OPP', u'WIN', u'LOSS', u'WL_1_25', u'WL_26_50',
       u'WL_51_100', u'WL_101_200', u'WL_201', u'ND1'],
      dtype='object')


**model data**

rpi_diff + team_rating_diff + A_RPI + B_RPI + win_pct_diff +
                A_RPI_C + B_RPI_C + TEAM_SEED_1 + TEAM_SEED + RD7_WIN + RD7_WIN_1 - 1

In [3]:
# load models
from sklearn.externals import joblib
decs_tree = joblib.load('models/larh_decision_t.model')
rand_f = joblib.load('models/larh_random_forest.model')
log_model = joblib.load('models/larh_logistic_reg.model')

In [4]:
team1 = raw_input()
team2 = raw_input()

Kansas
Connecticut


** real_data is used for first

rpi_diff + team_rating_diff + A_RPI + B_RPI + win_pct_diff +
                A_RPI_C + B_RPI_C + TEAM_SEED_1 + TEAM_SEED + RD7_WIN + RD7_WIN_1 - 1

In [5]:
def create_test_dataset(team1, team2):
    data = []
    data.append(team_info.loc[team1, 'RPI'] - team_info.loc[team2, 'RPI']) #RPI diff
    data.append(team_info.loc[team1, 'TEAM_RATING'] - team_info.loc[team2, 'TEAM_RATING']) #TEAM RATING
    data.append(team_info.loc[team1, 'RPI']) #A_RPI
    data.append(team_info.loc[team2, 'RPI']) #B_RPI
    data.append(team_info.loc[team1, 'PCT'] - team_info.loc[team2, 'PCT']) #TEAM RATING
    data.append(team_info.loc[team1, 'RPI_C']) #A_RPI_C
    data.append(team_info.loc[team2, 'RPI_C']) #B_RPI_C
    data.append(team_info.loc[team2, 'TEAM_SEED']) #TEAM_SEED 2
    data.append(team_info.loc[team1, 'TEAM_SEED']) #TEAM_SEED
    data.append(team_info.loc[team1, 'RD7_WIN']) # WIN_F 2
    data.append(team_info.loc[team2, 'RD7_WIN']) # WIN_F 1
    return data


test_X = create_test_dataset(team1, team2)


A


In [7]:
from patsy import dmatrices
import tensorflow as tf
#load neural net
df = pd.read_csv('train_data_x1.csv',header=0)
df['win_pct_diff'] = df['A_PCT'] - df['B_PCT']
df['rpi_diff'] = df['A_RPI'] - df['B_RPI']
df['team_rating_diff'] = df['TEAM_RATING'] - df['TEAM_RATING_1']
from keras.utils import np_utils

y, X = dmatrices('TEAMRESULT ~ rpi_diff + team_rating_diff + A_RPI + B_RPI + win_pct_diff +\
                A_RPI_C + B_RPI_C + TEAM_SEED_1 + TEAM_SEED + RD7_WIN + RD7_WIN_1 - 1',
                df, return_type='dataframe')

X = X.values
y = y.values

dimof_input = X.shape[1]
dimof_output = len(set(y.flat))

y_test = np.ravel(y)
# Set y categorical
y = np_utils.to_categorical(y, dimof_output)
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD

model = Sequential()

# Set constants

dimof_middle = 100
dropout = 0.5

verbose = 1
"""MLP for classification (0,1)"""

model.add(Dense(dimof_middle, input_dim=dimof_input, init="uniform", activation='relu' ))
model.add(Dropout(dropout))
model.add(Dense(dimof_middle, init="uniform", activation='relu' ))
model.add(Dropout(dropout))
model.add(Dense(dimof_output, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop')
countof_epoch = 10
batch_size = 64

model.fit(
    X, y,
    show_accuracy=True,
    batch_size=batch_size, nb_epoch=countof_epoch, verbose=verbose)

loss, accuracy = model.evaluate(X, y, show_accuracy=True, verbose=verbose)
print('loss: ', loss)
print('accuracy: ', accuracy)

Epoch 1/10
542/542 [==============================] - 0s - loss: 0.6691 - acc: 0.6365     
Epoch 2/10
542/542 [==============================] - 0s - loss: 0.6440 - acc: 0.6458     
Epoch 3/10
542/542 [==============================] - 0s - loss: 0.6291 - acc: 0.6568     
Epoch 4/10
542/542 [==============================] - 0s - loss: 0.6318 - acc: 0.6624     
Epoch 5/10
542/542 [==============================] - 0s - loss: 0.6227 - acc: 0.6421     
Epoch 6/10
542/542 [==============================] - 0s - loss: 0.6232 - acc: 0.6716     
Epoch 7/10
542/542 [==============================] - 0s - loss: 0.6278 - acc: 0.6587     
Epoch 8/10
542/542 [==============================] - 0s - loss: 0.6254 - acc: 0.6550     
Epoch 9/10
542/542 [==============================] - 0s - loss: 0.6242 - acc: 0.6568     
Epoch 10/10
542/542 [==============================] - 0s     
('loss: ', 0.61315785639840303)
('accuracy: ', 0.6771217716575989)


Using TensorFlow backend.


In [16]:
print X
print np.array([test_X])
model.predict_proba(np.array([test_X]))

[[  6.52000000e-02   9.80000000e+00   6.71300000e-01 ...,   1.00000000e+00
    1.90982710e-01   8.50876000e-03]
 [  7.95000000e-02   1.31900000e+01   6.71300000e-01 ...,   1.00000000e+00
    1.90982710e-01   4.04470000e-04]
 [  7.16000000e-02   8.99000000e+00   6.71300000e-01 ...,   1.00000000e+00
    1.90982710e-01   1.02074200e-02]
 ..., 
 [ -1.18100000e-01  -7.23000000e+00   4.75500000e-01 ...,   1.60000000e+01
    1.90000000e-07   1.03520000e-04]
 [ -1.35500000e-01  -1.54100000e+01   4.75500000e-01 ...,   1.60000000e+01
    1.90000000e-07   2.37548300e-02]
 [ -2.18500000e-01  -2.76100000e+01   4.52800000e-01 ...,   1.60000000e+01
    1.00000000e-08   1.90982710e-01]]
[[  9.42000000e-02   9.14000000e+00   6.71300000e-01   5.77100000e-01
    1.89300000e-01   6.88000000e-01   5.83600000e-01   9.00000000e+00
    1.00000000e+00   1.90982710e-01   3.27233000e-03]]
1/1 [==============================] - 0s


array([[ 0.2147706 ,  0.77345568]])